Biyoinformatik Projesi: Patojenik ve Non-Patojenik Organizma
1.Proje Fikrinin Doğuşu ve Veri Kümelerinin Hikayesi

Projemin fikri, NCBI sitesinde((https://www.ncbi.nlm.nih.gov/)) proje için fikir ararken, bir bakteri ismini inceleyip patojenik bir organizma olduğunu öğrenmemle ortaya çıktı. Organizmaları patojenik ve non-patojenik olarak ayırabilecek bir model oluşturma fikri aklıma geldi. Bu amaca yönelik olarak NCBI veritabanından ilk bağlamda toplamda 20 organizmaya ait genom dizilimini tek tek indirdim. Bu dizilimlerin 10’u patojenik, 10’u ise non-patojenik organizmalara aitti. Her birini FASTA formatında kayıt ederek kendi veri setimi hazırladım. Çalışma boyunca bu dizilimleri kullanarak bir makine öğrenimi modeli eğiterek yeni gelen dizilimlerin patojenik olup olmadığını tespit edebilecek bir algoritma geliştirdim.
(Kullandığım her bir organizma ismi ve dizilim linkleri ekteki pdf dosyasına eklendi.)


In [ ]:
pip install biopython

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from Bio import SeqIO
import os
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report


2.Kullanılan Teknikler ve Yöntemler:

FASTA Dosyalarını Okuma: Kodun ilk aşamasında, Bio.SeqIO modülü kullanılarak FASTA formatındaki dosyalar okundu. read_fasta_files fonksiyonu, belirtilen dizindeki tüm .fasta dosyalarını gezer ve içlerindeki dizilimleri okur. Bu sırada, dizilimde sadece 'A', 'C', 'G', 'T' karakterlerini filtreleyerek temiz bir dizi oluşturur.

In [ ]:
from Bio import SeqIO
import os

def dosyaoku(directory):
    sequences = []  # Dizileri sakladığımız liste
    for filename in os.listdir(directory): # Dizindeki tüm dosyaları gezer
        if filename.endswith(".fasta"):   # .fasta uzantılı dosyaları gezmesi için
            filepath = os.path.join(directory, filename)
            for record in SeqIO.parse(filepath, "fasta"):
                seq = str(record.seq)
                cleaned_seq = ''.join([base for base in seq if base in 'ACGT']) # Dizi içinde yalnızca 'A', 'C', 'G', 'T' bazlarını bırakır
                sequences.append(cleaned_seq) # Temizlenmiş diziyi listeye ekler
    return sequences


# Patojenik ve Non-Patojenik dizilimleri okur
pathogenic_sequences = dosyaoku('./Patojenik')
non_pathogenic_sequences = dosyaoku('./Non-Patojenik')




...


Etiketleme:Her bir dizilim, patojenik (‘1’) veya non-patojenik (‘0’) olarak etiketlendi. Bu etiketler, ileride modelin öğrenmesi için kullanılacak.

Veri Çerçevesi Oluşturma:Pandas kullanılarak dizilimler ve etiketlerden bir veri çerçevesi (‘DataFrame’) oluşturuldu.

In [ ]:
import pandas as pd

# Etiketleri ekler
pathogenic_labels = [1] * len(pathogenic_sequences)
non_pathogenic_labels = [0] * len(non_pathogenic_sequences)

# Dizilimleri ve etiketleri birleştirir
sequences = pathogenic_sequences + non_pathogenic_sequences
labels = pathogenic_labels + non_pathogenic_labels

# Veri çerçevesi oluşturur
df = pd.DataFrame({'sequence': sequences, 'label': labels})
df.head() #Datasetin bir önizlemesini görmemizi sağlar


,sequence,label
0,ATGAATCCAAGCCAAATACTTGAAAATTTAAAAAAAGAATTAAGTG...,1
1,ATGGATTTTCCCCGCGAATTTGATGTGATCGTCGTTGGTGGCGGTC...,1
2,GGCAGTCAGCGAGTTCTGGTTGTCCGGTATCGGAGCGCCTGCACTC...,1
3,ATGAACGAACTTAGCAGTGTGGATTGGTCAAGAGCGCAGTTTGCAT...,1
4,AGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATT...,1


Kümeler Halinde Çözümleme (k-mer): Dizilimler, k=6 olarak belirlenen uzunluktaki alt dizilere bölünerek analiz edildi. Bu alt dizilere k-mer denir ve biyoinformatik uygulamalarında yaygın olarak kullanılır.
CountVectorizer kullanılarak bu k-mer dizileri vektörlere dönüştürüldü. Modelin bu vektörleri işlemesi daha kolaydır.


In [ ]:
# k-mer hesaplama fonksiyonu
def kmer_count(seq, k=6):
    return [seq[i:i+k] for i in range(len(seq)-k+1)]

vectorizer = CountVectorizer(analyzer=lambda x: kmer_count(x)) # Dizilimleri vektörlere dönüştür
X = vectorizer.fit_transform(df['sequence'])
y = df['label']

print(y)


0     1
1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     1
10    1
11    1
12    1
13    1
14    1
15    1
16    1
17    1
18    1
19    1
20    1
21    1
22    1
23    1
24    1
25    1
26    1
27    1
28    1
29    1
30    0
31    0
32    0
33    0
34    0
35    0
36    0
37    0
38    0
39    0
40    0
41    0
42    0
43    0
44    0
45    0
46    0
47    0
48    0
49    0
50    0
51    0
52    0
53    0
54    0
55    0
56    0
57    0
58    0
59    0
Name: label, dtype: int64


Veri Bölme: Veri, %80 eğitim ve %20 test olarak ayrıldı.
Model: Random Forest algoritması kullanıldı.Bu algoritma ile ilgili çalışırken bu siteden yararlandım.(https://www.geeksforgeeks.org/random-forest-algorithm-in-machine-learning/)
Tahmin ve Değerlendirme: Modelin test setindeki performansı ölçüldü.
Doğruluk (‘accuracy’), çeşitli hata ölçütleriyle birlikte raporlandı.

Non-patojenik organizmaların sınıflandırılmasında hatasız çalışan model, patojenik organizmalarda nispeten daha düşük bir başarı elde etti.Model, genel olarak tatmin edici bir performans göstermektedir.


In [ ]:
# Veriyi eğitim ve test olarak ayırır
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modeli eğitir
rf_model = RandomForestClassifier(n_estimators=100)
rf_model.fit(X_train, y_train)

# Tahmin yapar
y_pred_rf = rf_model.predict(X_test)

# Sonuçlar
print("Accuracy:", accuracy_score(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))


Accuracy: 0.8333333333333334
              precision    recall  f1-score   support

           0       1.00      0.75      0.86         8
           1       0.67      1.00      0.80         4

    accuracy                           0.83        12
   macro avg       0.83      0.88      0.83        12
weighted avg       0.89      0.83      0.84        12



Bir önceki tablo şu şekildeydi:
Accuracy: 0.5
            precision   recall  f1-score    support

        0        0.50     0.50      0.50          2
        1        0.50     0.50      0.50          2
        
 accuracy                           0.50          4
macro avg        0.50     0.50      0.50          4
weighted avg     0.50     0.50      0.50          4

Daha sonra veri setini 10'ar organizma .fasta dosyasından her bir etiket için 10 dizilim olarak artırdım.
Son olarak tablo bu şekilde:
Accuracy: 0.8333333333333334
              precision    recall  f1-score   support

           0       1.00      0.75      0.86         8
           1       0.67      1.00      0.80         4

    accuracy                           0.83        12
   macro avg       0.83      0.88      0.83        12
weighted avg       0.89      0.83      0.84        12

In [ ]:
new_sequence="AGTTTGATCATGGCTCAGATTGAACGCTGGCGGCAGGCCTAACACATGCAAGTCGAGCGGCAGCACAAGTGAGTTTACTCATGAGGTGGCGAGCGGCGGACGGGTGAGTAATGCCTAGGGATCTGCCCAGTCGAGGGGGATAACAGTTGGAAACGACTGCTAATACCGCATACGCCCTACGGGGGAAAGAGGGGGACTTTCGGGCCTCTCGCGATTGGATGAACCTAGGTGGGATTAGCTAGTTGGTGAGGTAATGGCTCACCAAGGCGACGATCCCTAGCTGTTCTGAGAGGATGATCAGCCACACTGGGACTGAGACACGGCCCAGACTCCTACGGGAGGCAGCAGTGGGGAATATTGCACAATGGGGGAAACCCTGATGCAGCCATGCCGCGTGTGTGAAGAAGGCCTTCGGGTTGTAAAGCACTTTCAGTAGGGAGGAAAGGGTAAGTCCTAATACGACTTATCTGTGACGTTACCTACAGAAGAAGGACCGGCTAACTCCGTGCCAGCAGCCGCGGTAATACGGAGGGTCCGAGCGTTAATCGGAATTACTGGGCGTAAAGCGTGCGCAGGCGGTTTGTTAAGCGAGATGTGAAAGCCCTGGGCTCAACCTAGGAATCGCATTTCGAACTGACCAACTAGAGTCTTGTAGAGGGGGGTAGAATTCCAGGTGTAGCGGTGAAATGCGTAGAGATCTGGAGGAATACCGGTGGCGAAGGCGGCCCCCTGGACAAAGACTGACGCTCATGCACGAAAGCGTGGGGAGCAAACAGGATTAGATACCCTGGTAGTCCACGCCGTAAACGATGTCTACTCGGAGTTTGGTGTCTTGAACACTGGGCTCTCAAGCTAACGCATTAAGTAGACCGCCTGGGGAGTACGGCCGCAAGGTTAAAACTCAAATGAATTGACGGGGGCCCGCACAAGCGGTGGAGCATGTGGTTTAATTCGATGCAACGCGAAGAACCTTACCTACTCTTGACATCCACGGAAGACTGCAGAGATGCGGTTGTGCCTTCGGGAACCGTGAGACAGGTGCTGCATGGCTGTCGTCAGCTCGTGTTGTGAAATGTTGGGTTAAGTCCCGCAACGAGCGCAACCCCTATCCTTATTTGCCAGCACGTAATGGTGGGAACTCTAGGGAGACTGCCGGTGATAAACCGGAGGAAGGTGGGGACGACGTCAAGTCATCATGGCCCTTACGAGTAGGGCTACACACGTGCTACAATGGCGAGTACAGAGGGTTGCAAAGCCGCGAGGTGGAGCTAATCTCACAAAGCTCGTCGTAGTCCGGATTGGAGTCTGCAACTCGACTCCATGAAGTCGGAATCGCTAGTAATCGTGGATCAGAATGCCACGGTGAATACGTTCCCGGGCCTTGTACACACCGCCCGTCACACCATGGGAGTGGGCTGCAAAAGAAGTGGGTAGCTTAACCTTCGGGGGGGCGCTCACCACTTTGTGGTTCATGACTGGGGTGAAGTCGTAACAAGGTAGCCCTAGGGGAACCTGGGGCTGGATCACCTCCTTA"
# Deneme için verdiğim sequence patojenik olmayan bir organizmaya ait.
# Dizilimi sayısal hale getirir
new_sequence_numeric = vectorizer.transform([new_sequence])

# Tahmin yaparız
prediction = rf_model.predict(new_sequence_numeric)
prediction_proba = rf_model.predict_proba(new_sequence_numeric)

# Son olarak sonucu yazdırırız
if prediction[0] == 1:
    print("Bu dizilim patojeniktir.")
else:
    print("Bu dizilim patojenik değildir.")

print("Tahmin olasılığı:", prediction_proba)


Bu dizilim patojenik değildir.
Tahmin olasılığı: [[0.77 0.23]]
